In [ ]:
import git
import requests
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

In [ ]:
git_paths = {'PyNWB': "NeurodataWithoutBorders/pynwb", 
             'MatNWB': "NeurodataWithoutBorders/matnwb",
             'NWBWidgets': "NeurodataWithoutBorders/nwb-jupyter-widgets",
             'NWBInspector': "NeurodataWithoutBorders/nwbinspector",
             'Hackathons': "NeurodataWithoutBorders/nwb_hackathons",
             'NWB_Schema': "NeurodataWithoutBorders/nwb-schema",
             'NWB_Schema_Language': "NeurodataWithoutBorders/nwb-schema-language",
             'HDMF': 'hdmf-dev/hdmf',
             'HDMF_Common_Schema': 'hdmf-dev/hdmf-common-schema',
             'HDMF_DocUtils': 'hdmf-dev/hdmf-docutils',
             'HDMF Schema Language' : 'hdmf-dev/hdmf-schema-language',
             'NDX_Template': 'nwb-extensions/ndx-template',
             'NDX_Staged_Extensions': 'nwb-extensions/staged-extensions',
             #'NDX Webservices': 'https://github.com/nwb-extensions/nwb-extensions-webservices.git',
             'NDX_Catalog': 'nwb-extensions/nwb-extensions.github.io',
             'NDX_Extension_Smithy': 'nwb-extensions/nwb-extensions-smithy',
             'NWB_1.x_Matlab': 'NeurodataWithoutBorders/api-matlab',
             'NWB_1.x_Python': 'NeurodataWithoutBorders/api-python'
            }

In [ ]:
def get_releases(repo_name):
    """
    Get the last 100 release for the given repo
    
    :param repo_name: Name of the repo consisting of <owner>/<repo>, e.g., 'NeurodataWithoutBorders/pynwb'
    
    :returns: List of dicts with the release data
    """
    # NOTE: GitHub uses pageination. Here we set the number of items per page to 100
    #       which should usually fit all releases, but in the future we may need to
    #       iterate over pages to get all the releases not just the latests 100
    r = requests.get("https://api.github.com/repos/%s/releases?per_page=100" % (repo_name.lstrip('/').rstrip('/')))
    if not r.ok:
        r.raise_for_status()
    return r.json()

def get_release_names_and_dates(releases):
    """Given the output of get_releases get just the names and dates of releases"""
    names = []
    dates = []
    for rel in releases:
        if 'Latest' not in rel['name']:
            names.append(rel['tag_name'])
            dates.append(rel['published_at'])  
    dates = [datetime.strptime(d[0:10], "%Y-%m-%d") for d in dates]
    return names, dates

def plot_release_timeline(reponame, names, dates, figsize=None, fontsize=14, month_intervals=3, xlim=None, ax=None, title_on_yaxis=False):
    # Based on https://matplotlib.org/stable/gallery/lines_bars_and_markers/timeline.html
    # Choose some nice levels
    levels = np.tile([-5, 5, -3, 3, -1, 1],
                     int(np.ceil(len(dates)/6)))[:len(dates)]

    # Create figure and plot a stem plot with the date
    if ax is None:
        fig, ax = plt.subplots(figsize=(12,5) if figsize is None else figsize)
    
    ax.vlines(dates, 0, levels, color="dodgerblue")  # The vertical stems.
    ax.plot(dates, np.zeros_like(dates), "-o",
            color="k", markerfacecolor="w")  # Baseline and markers on it.

    # annotate lines
    for d, l, r in zip(dates, levels, names):
        ax.annotate(r, xy=(d, l),
                    xytext=(fontsize+2, np.sign(l)*3), textcoords="offset points",
                    horizontalalignment="right",
                    verticalalignment="bottom" if l > 0 else "top",
                    fontsize=fontsize)

    # format xaxis with 4 month intervals
    if xlim is not None:
        ax.set_xlim(xlim)
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=month_intervals))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
    plt.setp(ax.get_xticklabels(), rotation=30, ha="right", fontsize=fontsize)
        
    # set the title
    if title_on_yaxis:
        ax.get_yaxis().set_ticks([])
        ax.set_ylabel(reponame, fontsize=fontsize)
    else:  
        ax.set_title("%s release dates" % reponame, fontdict={'fontsize': fontsize})
        ax.yaxis.set_visible(False)

    ax.margins(y=0.1)
    return ax

In [ ]:
# Collect all releases
releases = {k: get_releases(v) for k, v in git_paths.items()}

In [ ]:
# Plot results for select repos
select_repos = ['PyNWB', 'HDMF', 'MatNWB', 'NWB_Schema', 'HDMF_Common_Schema']
fig, axes = plt.subplots(figsize=(16, len(select_repos)*4.2), nrows=len(select_repos), ncols=1, sharex=True, sharey=False, squeeze=True)
for i, repo in enumerate(select_repos):
    release_names, release_dates = get_release_names_and_dates(releases[repo])
    plot_release_timeline(reponame=repo.replace("_", " "), 
                          names=release_names, 
                          dates=release_dates,
                          #figsize=(15,4),
                          fontsize=14, 
                          month_intervals=2,
                          xlim=(datetime.strptime("2017-10-01", "%Y-%m-%d"), 
                                datetime.strptime("2021-10-01", "%Y-%m-%d"),),
                          ax=axes[i],
                          title_on_yaxis=True)
    #if i < len(axes) -1:
    #    axes[i].xaxis.set_visible(False)
    axes[i].grid(axis="x", linestyle='dashed', color='lightgray')
axes[0].set_title("Timeline of NWB Release", fontdict={'fontsize':16})
plt.subplots_adjust(wspace=0, hspace=0.0)
plt.savefig('nwb_software_releases_timeline.pdf', dpi=300)
plt.show()